<h1 align="center"> Lie point symmetries for first order ODEs </h1>
 <h3 align="center">A symbolic algorithm for the maxima CAS </h3>


#  
#   
 

# The method of undetermined coefficients

The method of undetermined coefficients (symmetry5) is the most successful methods of the paper of Cheb-terrab and Roche. In the paper, the symmetry generators $\xi$ and $\eta$ are assumed to be bivariate: $\xi=a_1x+a_2y+a_3$, $\eta=b_1x+b_2y+b_3$ and the coefficients $a_1..a_3,b_1..b_3$ are solved using the method of undetermined coefficients. The default in our maxima implementation is a bivariate polynomial of degree 2. For $\xi$ this means $\xi=a_1x + a_2y + a_3 + a_4x^2 + a_5xy + a_6y^2$. This means we can solve many more odes with the method of undetermined coefficients than mentioned in the paper. 


In [1]:
kill(all);

done

In [ ]:
batch("~/mathematics/maxima_files/kamke1_1.mac");

In [ ]:
batch("~/mathematics/maxima_files/ode1_lie.mac");

In [4]:
ode:kamke1[23];

'diff(y,x,1)+a*y^2-b

In [5]:
ode1Solve(ode,y,x,'useMethod="symmetry5",'returnSolution=false,'returnIntegratingFactor=false);

   trying [xi,eta]=[ax+by+c,fx+gy+h] 
s the symmetry is not valid) 
*** solution found : [xi,eta]=[ax+by+c,fx+gy+h] (symmetry5) *** 


[[b,b-a*y^2]]

The ODE has a polynomial symmetry of degree 2. 

Polynomial symmetries occur frequently and the method of undetermined coefficients is one of the most successfull algorithms for finding symmetries. 
The method of undetermined coefficients sees rational variables like log(), sqrt() and exp() as additional variables. A multivariate polynomial is constructed that includes these variables. For instance for Kamke ODE 1.59: 

In [6]:
ode:kamke1[59];

'diff(y,x,1)-a*sqrt(y^2+1)-b

This ODE has a square root term. First, we try to factor the term. In this case, we can factor into $\sqrt{y^2+1}=\sqrt{y+i}\sqrt{y-i}$. The factors are complex, and we value real over complex. This is why we ignore the complex factorization and introduce the new variable $z=\sqrt{y^2+1}$ and we construct a second degree polynomial in $x,y,z$. Note that if the ode had a term of the form $\sqrt{y^2-1}=\sqrt{y+1}\sqrt{y-1}$, we would have introduced 2 new variables $z_1=\sqrt{y+1}$ and $z_2=\sqrt{y-1}$. Factorization gives us more variables and a more general shape of the determining equations.

In [7]:
ode1Solve(ode,y,x,'useMethod="symmetry5");

   trying [xi,eta]=[ax+by+c,fx+gy+h] 
s the symmetry is not valid) 
*** solution found : [xi,eta]=[ax+by+c,fx+gy+h] (symmetry5) *** 
                                /
                                [         1
                                ]         2
                                / a sqrt(y  + 1) + b


[[a*sqrt(y^2+1)+a*y+x,(a^2*y+a*x+a)*sqrt(y^2+1)+a^2*y^2+a*b*y+b*x-b^2+b+a^2],
 [1/((b-1)*(a*sqrt(y^2+1)+b))],
 (x-'integrate(1/(a*sqrt(y^2+1)+b),y))/(b-1) = %c]

The method of undetermined coefficients finds a polynomial symmetry and an implicit solution (we asked for an explicit solution, hence the warning).


Additionally, the method of undetermined coefficients searches for rational polynomial symmetries of the form $\xi=Q_1/P_1$ and $\eta=Q_2/P_2$, with $P,Q$ bivariate polynomials. Because the search for such symmetries is much more expensive, the default is to search only for rational symmetries of the form $[\xi,\eta]=[0,\frac{Q}{P}]$ and $[\xi,\eta]=[\frac{Q}{P},0]$. It turns out that there is a class of ODEs of the form $y'=\frac{Q(x,y)}{P(x,y)}$ and $y'=\frac{Q(x,y)}{xP(x,y)}$ whose symmetries have the form $[\xi,\eta]=[0,\frac{F(x,y)}{P(x,y)}]$, so symmetries whose denominator is the denominator (or denominator times $x$) of the right-hand side of the differential equation. We are also looking for these symmetries. 

In [8]:
ode:kamke1[346];

x*'diff(y,x,1)*(y*log(x*y)+y-a*x)-y*(a*x*log(x*y)-y+a*x)

This ODE can be written as $y'=Q/P$, with $P=x(\log(xy)+y-ax)$. So we are factoring out the $x$, and are looking for symmetries of the form $[\xi,\eta] =[0,\frac{F(x,y)}{\log(xy)+y-ax}]$ symmetry5 is automatically looking for such symmetries.

In [9]:
ode1Solve(ode,y,x,'useMethod="symmetry5");

   trying [xi,eta]=[ax+by+c,fx+gy+h] 
s the symmetry is not valid) 
s the symmetry is not valid) 
*** solution found : [xi,eta]=[0,(fx+gy+h)/P(x,y)] (symmetry5) *** 


[[0,y/(y*log(y)+(log(x)+1)*y-a*x)],[-1/(x*y)],
 [y = (a*x*log(x*y)+%c)/log(x*y)]]

note that the denominator of the symmetry does not match the original denominator of the ODE. This is because internally, ode1Solve tries to simplify the ODE by rewriting $\log(xy)$ in terms of $\log(x)$ and $\log(y)$. These canonical forms can be obtained as follows:

In [10]:
ode1CanonicalForm(ode,y,x);

'diff(y,x,1) = (a*x*y*log(y)-y^2+(a*x*log(x)+a*x)*y)
             /(x*y*log(y)+(x*log(x)+x)*y-a*x^2)

In [11]:
P:factor_list(denom(rhs(%)));

[x,y*log(y)+log(x)*y+y-a*x]

# user defined functions for symmetries

symmetry5 can be used to look for user defined shapes of symmetries. The user has to supply a function with unknown coefficients for the symmetries $\xi,\eta$. Let's take a look at Kamke ODE 1.2, which cannot be solved automatically by ode1Solve:

In [12]:
ode:kamke1[62];

'diff(y,x,1) = (y-x^2*sqrt(x^2-y^2))/(x*y*sqrt(x^2-y^2)+x)

In [13]:
returnSolution:false;

false

In [14]:
returnIntegratingFactor:false;

false

In [15]:
ode1Solve(ode,y,x);

   trying y' = F(y) ... 
   trying y' = F(x) ... 
   trying y' = f(x)*g(y) ... 
   trying exact ... 
   trying linear ... 
   trying inverse linear ... 
   trying Bernoulli ... 
   trying Riccati ... 
   trying simple symmetries ... 
   trying [xi,eta]=[F(x),G(y)] (non-general) 
   trying [xi,eta]=[ax+by+c,fx+gy+h] 
s the symmetry is not valid) 
s the symmetry is not valid) 
s the symmetry is not valid) 
expt: undefined: 0 to a negative exponent.
s the symmetry is not valid) 
expt: undefined: 0 to a negative exponent.
   trying [xi,eta]=[F(x),P(x)y+Q(x)] 
I is not linear in y. Just trying without transformation  
** trying inverse ode **  
** trying inverse ode **  
   trying y' = F(y) ... 
   trying y' = F(x) ... 
   trying y' = f(x)*g(y) ... 
   trying exact ... 
   trying linear ... 
   trying inverse linear ... 
   trying Bernoulli ... 
   trying Riccati ... 
   trying simple symmetries ... 
   trying [xi,eta]=[F(x),G(y)] (non-general) 
   trying [xi,eta]=[ax+by+c,fx+gy+h] 
s the s

[false,"no symmetries found"]

The ode contains the term $\sqrt{x^2-y^2}$. If the symmetry contains such a term, it will not be recognized by any of the methods. With the method of undetermined coefficients, we can look for a symmetry containing this term. Let's define a shortcut:

In [16]:
V1:sqrt(x^2-y^2);

sqrt(x^2-y^2)

Let's also define a shortcut for $\sqrt{x^2+y^2}$:

In [17]:
V2:sqrt(x^2+y^2);

sqrt(y^2+x^2)

We could define a very general multivariate polynomial with the variables $x,y,V_1,V_2$, but after some trial and erro you will find that this one works:

In [18]:
ode1Solve(kamke1[62],y,x,'useMethod="symmetry5",'xi=a1*x/V1^2+a2*x/V2^2,\eta=b1*y/V1^2+b2*y/V2^2);

   trying [xi,eta]=[ax+by+c,fx+gy+h] 
s the symmetry is not valid) 
*** solution found : [xi,eta]=[ax+by+c,fx+gy+h] (symmetry5) *** 


[[x/(y^2+x^2),y/(y^2+x^2)]]

we could have tried a symmetry search for a general rational symmetry of degree 2. However, this computation is very expensive. The search for such symmetries is not done automatically.

# Bibliography
[1] E.S. Cheb-Terrab and T. Kolokolnikov, First-order ordinary differential equations, symmetries and linear transformations, Euro. J. of Applied Mathematics 14 (2003)

[2] E.S. Cheb-Terrab and A.D. Roche, Symmetries and first order ODE patterns, Computer Physics Communications 113 (1998)

[3] E.S. Cheb-Terrab, L.G.S. Duarte and L.A.C.P. da Mota, Computer algebra solving of first order ODEs using symmetry methods, Computer Physics Communications 101 (1997)

[4] F. Schwarz, Symmetry analysis of Abel's equation, Studies in applied mathematics 100 (1998)

[5] F. Schwarz, Algorithmic solution of Abel's equation, Computing 61 (1998)

[6] E. Kamke, Differentialgleichungen, L$\mathrm{\ddot o}$sungsmethoden und L$\mathrm{\ddot o}$sungen, Leipzig (1959)
